In [2]:
def big_data(X,y,reqlen):
    Big_X = np.array(X)
    Big_Y = np.array(y)
    while len(Big_X) < reqlen:
        Big_X = np.concatenate((Big_X, Big_X))
        Big_Y = np.concatenate((Big_Y, Big_Y))
    print(Big_X.shape)
    indices = np.random.choice(Big_X.shape[0], reqlen, replace=False)
    new_X = Big_X[indices]
    new_Y = Big_Y[indices]
    return new_X, new_Y

# 1000 samples

In [5]:
from sklearn.datasets import load_boston
import sklearn.ensemble
import numpy as np
from sklearn.model_selection import train_test_split
from mosaicml import *
from mosaicml.constants import MLModelFlavours

import pandas as pd
from pandas import read_csv, DataFrame
from sklearn.preprocessing import Imputer
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from mosaicml.api import graph

# Importing the dataset
boston = load_boston()

# creating the model object
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=10)

X = boston.data
Y = boston.target

Big_X, Big_Y = big_data(X, Y,1000)

print(Big_X.shape)
print(Big_Y.shape)

train, test, labels_train, labels_test = train_test_split(Big_X, Big_Y, test_size=0.2, random_state=0)

rf.fit(train, labels_train)


# @scoring_func
def score(model, request):
    payload = request.json["payload"]
    data_list = payload
    data_array = np.asarray(data_list)
    try:
        prediction = model.predict(data_array)
    except:
        prediction = model.predict(data_array.reshape(1, -1))
    return prediction.tolist()

# to verify if score func works
import requests
req = requests.Request()
req.json = {"payload":train[10].tolist()}

score(rf,req)

from datetime import datetime
st = datetime.utcnow()
print(st,"\n")

y_pred=rf.predict(test)

register_model(rf,
                score,
                "boston_regression_1k",
                "boston_regression_1k",
                MLModelFlavours.sklearn,
                init_script="mkdir abc \\n cd abc \\n touch abc.txt",
                input_type="json",
                explain_ai=True,
               x_train=train,
               y_train=labels_train, 
               y_pred=y_pred,
               y_true=labels_test,
               features=list(boston.feature_names),
               original_features=list(boston.feature_names),
               model_type="regression",
               feature_names=list(boston.feature_names), 
               feature_ids=list(boston.feature_names), 
               target_names=['Housing_Price']
              )

(1012, 13)
(1000, 13)
(1000,)
2020-06-23 10:41:32.090732 



2020-06-23 10:41:37,562 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progressbar=False
2020-06-23 10:41:37,564 - skater.core.explanations - WARNING - Multiprocessing failed, going single process


[30/30] grid cells ████████████████████ Time elapsed: 1 seconds

2020-06-23 10:41:39,241 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progressbar=False
2020-06-23 10:41:39,243 - skater.core.explanations - WARNING - Multiprocessing failed, going single process


[26/26] grid cells ████████████████████ Time elapsed: 1 seconds

...

[30/30] grid cells ████████████████████ Time elapsed: 1 seconds

2020-06-23 10:41:59,912 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progress_bar=False
2020-06-23 10:41:59,914 - skater.core.explanations - WARNING - Multiprocessing failed, going single process


[13/13] features ████████████████████ Time elapsed: 0 seconds

2020-06-23 10:42:01,182 - skater.core.global_interpretation.tree_surrogate - INFO - post pruning applied ...
2020-06-23 10:42:01,197 - skater.core.global_interpretation.tree_surrogate - INFO - Scorer used MAE
2020-06-23 10:42:01,199 - skater.core.global_interpretation.tree_surrogate - INFO - original score using base model 0.6607262861370011
2020-06-23 10:42:01,254 - skater.core.global_interpretation.tree_surrogate - INFO - Summary: childrens of the following nodes are removed [50, 178]
2020-06-23 10:42:01,255 - skater.core.global_interpretation.tree_surrogate - INFO - Done generating prediction using the surrogate, shape (800,)
2020-06-23 10:42:01,257 - skater.core.global_interpretation.tree_surrogate - INFO - Done scoring, surrogate score 0.671; oracle score 0.471


2020-06-23 10:42:01.983297 

0:00:29.892565


In [ ]:

en = datetime.utcnow()
print(en,"\n")
diff = en - st
print(str(diff))

